In [ ]:
# Load Libraries
import pandas as pd
import numpy as np
import os
from scipy.io import loadmat
import matplotlib.pyplot as plt
import time
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense
from tensorflow.keras import optimizers
from sklearn.metrics import mean_squared_error
from tensorflow.python.keras.layers import Lambda
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from keras.models import Sequential, Model
from tensorflow.keras.optimizers import RMSprop, Adam
from keras.layers import Dense, LSTM, TimeDistributed, Flatten, Bidirectional, Input
from sklearn.preprocessing import MinMaxScaler
import h5py
import math
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from google.colab import files
from numpy import savetxt

In [ ]:
# Seed
np.random.seed(0)
tf.random.set_seed(0)

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Change the current working directory
os.chdir('/content/drive/My Drive/Colab Notebooks/PINN_Active_Passive_Paper/p5759')
# Print the current working directory
print("Current working directory: {0}".format(os.getcwd()))

Current working directory: /content/drive/My Drive/Colab Notebooks/PINN_Active_Passive_Paper/p5759


In [ ]:
# Function: Create Data History
def create_timesteps(data, n_steps):
	x = []
	y = []
	for i in range(len(data)-1):
		end_ix = i + n_steps
		if end_ix > len(data)-1:
			break
		x1, y1 = data[i:end_ix, :-1], data[end_ix, -1]
		x.append(x1)
		y.append(y1)
	return np.array(x), np.array(y)

In [ ]:
# Load Features
os.chdir('/content/drive/My Drive/Colab Notebooks/PINN_Active_Passive_Paper/p5759/Data')
Data1 = loadmat('p5759_Inputs.mat')
Data2 = loadmat('p5759_Outputs.mat')
Features = pd.DataFrame({'Time':Data1['Time'].ravel().round(2),'K':Data1['K'].ravel().round(2),'V':Data1['V'].ravel().round(2),'AE':Data1['AE'].ravel().round(2),
                         'ln_V0_Theta':Data1['ln_V0_Theta'].ravel().round(2),'Amp':Data1['Amp'].ravel().round(2)})
Output = pd.DataFrame({'Time':Data2['Time'].ravel().round(2),'SS':Data2['SS'].ravel().round(2),'ln_V_V0':Data2['ln_V_V0'].ravel().round(2),'TTF':Data2['TTF'].ravel().round(2),
                       'physics_out':Data2['physics_out'].ravel().round(2),'SR':Data2['SR'].ravel().round(2)})
df = Features.merge(Output, on = 'Time')
df = df[['K','V','AE','Amp','ln_V0_Theta','SS','TTF','ln_V_V0','SR','physics_out','Time']]
df_AE = df[['AE','AE']]
df_SS = df[['K','V','AE','Amp','SS']]
df_TTF = df[['K','V','AE','Amp','TTF']]
df_SR = df[['K','V','AE','Amp','ln_V_V0']]
df_SR_Org = df[['K','V','AE','Amp','SR']]
df_A = df[['K','V','AE','Amp','ln_V0_Theta']]
df_PhyOut = df[['K','V','AE','Amp','physics_out']]
xdf = df[['K','V','AE','Amp']]
ydf = df[['SS','TTF','ln_V_V0']]
print("Input Data:\n", xdf)
print("Target Data:\n",ydf)

Input Data:
                 K      V     AE       Amp
0      3511910.82  23.33   1.50  12313.25
1      3511805.49  23.16   1.67  12390.65
2         3807.30  16.44   1.67  12433.71
3          992.68  16.27   1.33  12425.59
4          999.61  16.27   1.33  12417.39
...           ...    ...    ...       ...
2549  46495304.45  57.63  21.00  14054.81
2550  28519703.48  45.56  28.00  14150.11
2551  28519724.49  45.56  26.67  14268.53
2552  28519838.19  45.57   8.67  14322.32
2553  28519979.30  45.56   2.00  14373.05

[2554 rows x 4 columns]
Target Data:
          SS   TTF  ln_V_V0
0     11.43  0.00   -11.51
1     11.46  5.78   -11.51
2     11.54  5.67   -11.51
3     11.65  5.56   -11.51
4     11.73  5.46   -11.51
...     ...   ...      ...
2549  12.79  0.00   -11.51
2550  11.50  0.00     1.45
2551  11.33  0.00   -11.51
2552  11.41  0.00   -11.51
2553  11.52  0.00   -11.51

[2554 rows x 3 columns]


In [ ]:
# Preprocessing (AE)
arr = df_AE.to_numpy()
n_steps = 100
xdf_AE, ydf_AE = create_timesteps(arr, n_steps)

# Preprocessing (SS)
arr = df_SS.to_numpy()
n_steps = 100
xdf, ydf_SS = create_timesteps(arr, n_steps)

# Preprocessing (TTF)
arr = df_TTF.to_numpy()
n_steps = 100
xdf, ydf_TTF = create_timesteps(arr, n_steps)

# Preprocessing (SR)
arr = df_SR.to_numpy()
n_steps = 100
xdf, ydf_SR = create_timesteps(arr, n_steps)

# Preprocessing (SR_Org)
arr = df_SR_Org.to_numpy()
n_steps = 100
xdf, ydf_SR_Org = create_timesteps(arr, n_steps)

# Preprocessing (A)
arr = df_A.to_numpy()
n_steps = 100
xdf, Adf = create_timesteps(arr, n_steps)

# Preprocessing (Physics_Out)
arr = df_PhyOut.to_numpy()
n_steps = 100
xdf, phy_outdf = create_timesteps(arr, n_steps)

print('Features shape, X = ', np.shape(xdf))
print('Target shape (SS), Y = ', np.shape(ydf_SS))
print('Target shape (TTF), Y = ', np.shape(ydf_TTF))
print('Target shape (SR), Y = ', np.shape(ydf_SR))
print('Target shape (AE), Y = ', np.shape(ydf_AE))
print('Target shape (A), Y = ', np.shape(Adf))
print('Target shape (Physics_Out), Y = ', np.shape(phy_outdf))

# Reshape
in_dim = xdf.shape[1]*xdf.shape[2]
xdf = xdf.reshape((xdf.shape[0], in_dim))
print('After reshaping, X = ', np.shape(xdf))

# Reshape AE
in_dim = xdf_AE.shape[1]*xdf_AE.shape[2]
xdf_AE = xdf_AE.reshape((xdf_AE.shape[0], in_dim))
print('After reshaping, xdf_AE = ', np.shape(xdf_AE))

Features shape, X =  (2454, 100, 4)
Target shape (SS), Y =  (2454,)
Target shape (TTF), Y =  (2454,)
Target shape (SR), Y =  (2454,)
Target shape (AE), Y =  (2454,)
Target shape (A), Y =  (2454,)
Target shape (Physics_Out), Y =  (2454,)
After reshaping, X =  (2454, 400)
After reshaping, xdf_AE =  (2454, 100)


In [ ]:
# Split into train-val-test (SS)
x_train, x_test, y_train_SS, y_test_SS = train_test_split(xdf, ydf_SS, test_size=0.2, shuffle=False)
X_train, X_val, Y_train_SS, Y_val_SS = train_test_split(x_train, y_train_SS, test_size=0.125, shuffle=False)

# Split into train-val-test (TTF)
x_train, x_test, y_train_TTF, y_test_TTF = train_test_split(xdf, ydf_TTF, test_size=0.2, shuffle=False)
X_train, X_val, Y_train_TTF, Y_val_TTF = train_test_split(x_train, y_train_TTF, test_size=0.125, shuffle=False)

# Split into train-val-test (SR)
x_train, x_test, y_train_SR, y_test_SR = train_test_split(xdf, ydf_SR, test_size=0.2, shuffle=False)
X_train, X_val, Y_train_SR, Y_val_SR = train_test_split(x_train, y_train_SR, test_size=0.125, shuffle=False)

# Split into train-val-test (SR_Org)
x_train, x_test, y_train_SR_Org, y_test_SR_Org = train_test_split(xdf, ydf_SR_Org, test_size=0.2, shuffle=False)
X_train, X_val, Y_train_SR_Org, Y_val_SR_Org = train_test_split(x_train, y_train_SR_Org, test_size=0.125, shuffle=False)

# Split into train-val-test (AE)
x_train, x_test, y_train_AE, y_test_AE = train_test_split(xdf, xdf_AE, test_size=0.2, shuffle=False)
X_train, X_val, Y_train_AE, Y_val_AE = train_test_split(x_train, y_train_AE, test_size=0.125, shuffle=False)

# Split into train-val-test (ae)
x_train, x_test, y_train_ae, y_test_ae = train_test_split(xdf, ydf_AE, test_size=0.2, shuffle=False)
X_train, X_val, Y_train_ae, Y_val_ae = train_test_split(x_train, y_train_ae, test_size=0.125, shuffle=False)

# Split into train-val-test (A)
x_train, x_test, y_train_A, y_test_A = train_test_split(xdf, Adf, test_size=0.2, shuffle=False)
X_train, X_val, Y_train_A, Y_val_A = train_test_split(x_train, y_train_A, test_size=0.125, shuffle=False)

# Split into train-val-test (Physics Out)
x_train, x_test, y_train_Phy, y_test_Phy = train_test_split(xdf, phy_outdf, test_size=0.2, shuffle=False)
X_train, X_val, Y_train_Phy, Y_val_Phy = train_test_split(x_train, y_train_Phy, test_size=0.125, shuffle=False)

# Select Train
X_train = X_train[244:735]
Y_train_SS = Y_train_SS[244:735]
Y_train_TTF = Y_train_TTF[244:735]
Y_train_SR = Y_train_SR[244:735]
Y_train_SR_Org = Y_train_SR_Org[244:735]
Y_train_AE = Y_train_AE[244:735]
Y_train_ae = Y_train_ae[244:735]
Y_train_Phy = Y_train_Phy[244:735]
Y_train_A = Y_train_A[244:735]

# Normalize the input
scaler = MinMaxScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train))
X_val = pd.DataFrame(scaler.transform(X_val))
x_test = pd.DataFrame(scaler.transform(x_test))

print("Training samples:", np.shape(X_train)[0])
print("Validation samples:", np.shape(X_val)[0])
print("Testing samples:", np.shape(x_test)[0])

Training samples: 491
Validation samples: 246
Testing samples: 491


In [ ]:
# Normalize the output: SS
scaler_ss = MinMaxScaler(feature_range=(0, 1))
Y_train_SS = pd.DataFrame(scaler_ss.fit_transform(Y_train_SS.reshape(-1, 1)))
Y_val_SS = pd.DataFrame(scaler_ss.transform(Y_val_SS.reshape(-1, 1)))
y_test_SS = pd.DataFrame(scaler_ss.transform(y_test_SS.reshape(-1, 1)))

In [ ]:
# Normalize the output: TTF
scaler_ttf = MinMaxScaler(feature_range=(0, 1))
Y_train_TTF = pd.DataFrame(scaler_ttf.fit_transform(Y_train_TTF.reshape(-1, 1)))
Y_val_TTF = pd.DataFrame(scaler_ttf.transform(Y_val_TTF.reshape(-1, 1)))
y_test_TTF = pd.DataFrame(scaler_ttf.transform(y_test_TTF.reshape(-1, 1)))

In [ ]:
# # Normalize the output: SR
scaler_sr = MinMaxScaler(feature_range=(-1, 1))
Y_train_SR = pd.DataFrame(scaler_sr.fit_transform(Y_train_SR.reshape(-1, 1)))
Y_val_SR = pd.DataFrame(scaler_sr.transform(Y_val_SR.reshape(-1, 1)))
y_test_SR = pd.DataFrame(scaler_sr.transform(y_test_SR.reshape(-1, 1)))

In [ ]:
# # Normalize the output: SR_Org
scaler_sr_org = MinMaxScaler(feature_range=(0, 1))
Y_train_SR_Org = pd.DataFrame(scaler_sr_org.fit_transform(Y_train_SR_Org.reshape(-1, 1)))
Y_val_SR_Org = pd.DataFrame(scaler_sr_org.transform(Y_val_SR_Org.reshape(-1, 1)))
y_test_SR_Org = pd.DataFrame(scaler_sr_org.transform(y_test_SR_Org.reshape(-1, 1)))

In [ ]:
# Normalize the input: AE
scaler_ae = MinMaxScaler(feature_range=(0, 1))
Y_train_AE = pd.DataFrame(scaler_ae.fit_transform(Y_train_AE))
Y_val_AE = pd.DataFrame(scaler_ae.transform(Y_val_AE))
y_test_AE = pd.DataFrame(scaler_ae.transform(y_test_AE))

In [ ]:
# Normalize Amp
scaler_A = MinMaxScaler(feature_range=(0, 1))
Y_train_A = pd.DataFrame(scaler_A.fit_transform(Y_train_A.reshape(-1, 1)))
Y_val_A = pd.DataFrame(scaler_A.transform(Y_val_A.reshape(-1, 1)))
y_test_A = pd.DataFrame(scaler_A.transform(y_test_A.reshape(-1, 1)))

In [ ]:
# Define Physics 1
# Rate and State Friction
# mu  = mu0 + a*ln(V/V0)+ b*ln(V0*Theta/Dc)
# SS = C1 + C2ln(V/V0) + C3ln(V0*Theta/Dc)

class gradient_1(tf.keras.layers.Layer):
  def __init__(self):
    super(gradient_1,self).__init__()
    self.SS = tf.keras.layers.Dense(1)
    self.SR = tf.keras.layers.Dense(1)
    self.A = tf.keras.layers.Dense(1)
    self.all_scale = tf.keras.layers.Dense(1)
    self.c1 = self.add_weight("c1",trainable=True)
    self.c2 = self.add_weight("c2",trainable=True)
    self.c3 = self.add_weight("c3",trainable=True)
    self.c4 = self.add_weight("c4",trainable=True)
  def call(self, params):
    ss_pred, sr_pred, Amp = params
    ss_pred = self.SS(ss_pred)
    sr_pred = self.SR(sr_pred)
    Amp = self.A(Amp)

    term1 =  ss_pred
    term2 =  sr_pred
    term3 =  Amp - self.c4

    term4 = tf.multiply(self.c2,term2)
    term5 = tf.multiply(self.c3,term3)

    eqn = term1-self.c1-term4-term5
    eqn = self.all_scale(eqn)

    return eqn

In [ ]:
# PINN 2
# AE_Rate = C1 * SR

class gradient_2(tf.keras.layers.Layer):
  def __init__(self):
    super(gradient_2,self).__init__()
    self.SRORG = tf.keras.layers.Dense(1)
    self.AE = tf.keras.layers.Dense(1)
    self.all_scale = tf.keras.layers.Dense(1)
    self.c5 = self.add_weight("c5",trainable=True)

  def call(self, params):
    sr_org_pred, ae = params
    sr_org_pred = self.SRORG(sr_org_pred)
    ae = self.AE(ae)

    term1 = ae
    term2 = self.c5*sr_org_pred
    eqn = term1-term2
    eqn = self.all_scale(eqn)

    return eqn

In [ ]:
# Define PINN
def MLP_Physics():

  # Define inputs
  X_train = layers.Input(shape=(400),name='X_train')
  AE =layers.Input(shape=(100),name='AE')
  Amp =layers.Input(shape=(1),name='Amp')
  ae = layers.Input(shape=(1),name='ae')

  # Prediction: Shear Stress + TTF
  x = layers.Dense(128,kernel_initializer = 'normal', activation="relu")(X_train)
  x = layers.Dense(64,kernel_initializer = 'normal', activation="relu")(x)
  x = layers.Dense(32,kernel_initializer = 'normal', activation="relu")(x)
  x = layers.Dense(16, kernel_initializer = 'normal', activation="relu")(x)
  x = layers.Dense(8, kernel_initializer = 'normal', activation="relu")(x)

  ss_pred = layers.Dense(1,kernel_initializer = 'normal',activation="linear")(x)
  ttf_pred = layers.Dense(1,kernel_initializer = 'normal',activation="linear")(x)

  # Prediction: SR
  x = layers.Dense(128,kernel_initializer = 'normal', activation="relu")(AE)
  x = layers.Dense(64,kernel_initializer = 'normal', activation="relu")(x)
  x = layers.Dense(32,kernel_initializer = 'normal', activation="relu")(x)
  x = layers.Dense(16, kernel_initializer = 'normal', activation="relu")(x)
  x = layers.Dense(8, kernel_initializer = 'normal', activation="relu")(x)
  sr_pred = layers.Dense(1,kernel_initializer = 'normal',activation="linear")(x)
  sr_org_pred = layers.Dense(1,kernel_initializer = 'normal',activation="linear")(x)

  grad_out_1 = gradient_1()([ss_pred,sr_pred,Amp])
  grad_out_2 = gradient_2()([sr_org_pred,ae])
  model = keras.Model(inputs=[X_train,AE,Amp,ae],outputs=[ss_pred,ttf_pred,sr_pred,sr_org_pred,grad_out_1,grad_out_2])
  return model

In [ ]:
# Model Summary
model=MLP_Physics()
model.compile(loss='mse', optimizer=tf.keras.optimizers.legacy.Adam(1e-3), metrics=['mse'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 X_train (InputLayer)        [(None, 400)]                0         []                            
                                                                                                  
 AE (InputLayer)             [(None, 100)]                0         []                            
                                                                                                  
 dense (Dense)               (None, 128)                  51328     ['X_train[0][0]']             
                                                                                                  
 dense_7 (Dense)             (None, 128)                  12928     ['AE[0][0]']                  
                                                                                              

In [ ]:
# Train The Model
start_time = time.time()
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=20,verbose=1, mode='auto')
callbacks_list = [earlystop]
history = model.fit([X_train,Y_train_AE,Y_train_A,Y_train_ae],[Y_train_SS,Y_train_TTF,Y_train_SR,Y_train_SR_Org,Y_train_Phy,Y_train_Phy], epochs=100, batch_size=32,callbacks=callbacks_list,
                     validation_data=([X_val,Y_val_AE,Y_val_A,Y_val_ae],[Y_val_SS,Y_val_TTF,Y_val_SR,Y_val_SR_Org,Y_val_Phy,Y_val_Phy]), verbose=1)
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='val')
plt.legend()
plt.show()
end_time = time.time()
print("---Training time: %0.8f seconds ---" % (end_time - start_time))

Train on 491 samples, validate on 246 samples
Epoch 1/100
480/491 [============================>.] - ETA: 0s - loss: 6.6838 - dense_5_loss: 0.4572 - dense_6_loss: 0.2926 - dense_12_loss: 0.9831 - dense_13_loss: 0.0176 - gradient_1_loss: 0.0334 - gradient_2_loss: 4.8998 - dense_5_mse: 0.4572 - dense_6_mse: 0.2926 - dense_12_mse: 0.9831 - dense_13_mse: 0.0176 - gradient_1_mse: 0.0334 - gradient_2_mse: 4.8998

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training_v1.py:2335: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


491/491 [==============================] - 4s 7ms/sample - loss: 6.7686 - dense_5_loss: 0.4521 - dense_6_loss: 0.2925 - dense_12_loss: 0.9817 - dense_13_loss: 0.0165 - gradient_1_loss: 0.0328 - gradient_2_loss: 5.1446 - dense_5_mse: 0.4554 - dense_6_mse: 0.2926 - dense_12_mse: 0.9826 - dense_13_mse: 0.0172 - gradient_1_mse: 0.0332 - gradient_2_mse: 4.9876 - val_loss: 4.8392 - val_dense_5_loss: 0.4679 - val_dense_6_loss: 0.3251 - val_dense_12_loss: 0.9597 - val_dense_13_loss: 0.0202 - val_gradient_1_loss: 0.4960 - val_gradient_2_loss: 2.4766 - val_dense_5_mse: 0.4691 - val_dense_6_mse: 0.3214 - val_dense_12_mse: 0.9598 - val_dense_13_mse: 0.0210 - val_gradient_1_mse: 0.4958 - val_gradient_2_mse: 2.5720
Epoch 2/100
491/491 [==============================] - 0s 449us/sample - loss: 5.9753 - dense_5_loss: 0.3797 - dense_6_loss: 0.1994 - dense_12_loss: 0.9093 - dense_13_loss: 0.0208 - gradient_1_loss: 0.0181 - gradient_2_loss: 4.2581 - dense_5_mse: 0.3840 - dense_6_mse: 0.2052 - dense_12_ms

In [ ]:
# Predictions: Training
y_predtrain = model.predict([X_train,Y_train_AE,Y_train_A,Y_train_ae])
y_predtrain_SS = np.array(y_predtrain)[0,]
y_predtrain_TTF =np.array(y_predtrain)[1,]
y_predtrain_SR =np.array(y_predtrain)[2,]
y_predtrain_SR_Org =np.array(y_predtrain)[3,]

# Predictions: Validation
y_predval = model.predict([X_val,Y_val_AE,Y_val_A,Y_val_ae])
y_predval_SS = np.array(y_predval)[0,]
y_predval_TTF =np.array(y_predval)[1,]
y_predval_SR =np.array(y_predval)[2,]
y_predval_SR_Org =np.array(y_predval)[3,]

# Predictions: Testing
y_predtest = model.predict([x_test,y_test_AE,y_test_A,y_test_ae])
y_predtest_SS = np.array(y_predtest)[0,]
y_predtest_TTF =np.array(y_predtest)[1,]
y_predtest_SR =np.array(y_predtest)[2,]
y_predtest_SR_Org =np.array(y_predtest)[3,]

In [ ]:
# Inverse scaling: SS
Y_train_SS = scaler_ss.inverse_transform(Y_train_SS)
Y_val_SS = scaler_ss.inverse_transform(Y_val_SS)
y_test_SS = scaler_ss.inverse_transform(y_test_SS)
y_predtrain_SS = scaler_ss.inverse_transform(y_predtrain_SS)
y_predval_SS = scaler_ss.inverse_transform(y_predval_SS)
y_predtest_SS = scaler_ss.inverse_transform(y_predtest_SS)

In [ ]:
# Inverse scaling: TTF
Y_train_TTF = scaler_ttf.inverse_transform(Y_train_TTF)
Y_val_TTF = scaler_ttf.inverse_transform(Y_val_TTF)
y_test_TTF = scaler_ttf.inverse_transform(y_test_TTF)
y_predtrain_TTF = scaler_ttf.inverse_transform(y_predtrain_TTF)
y_predval_TTF = scaler_ttf.inverse_transform(y_predval_TTF)
y_predtest_TTF = scaler_ttf.inverse_transform(y_predtest_TTF)

In [ ]:
# Inverse scaling: SR
Y_train_SR = scaler_sr.inverse_transform(Y_train_SR)
Y_val_SR = scaler_sr.inverse_transform(Y_val_SR)
y_test_SR = scaler_sr.inverse_transform(y_test_SR)
y_predtrain_SR = scaler_sr.inverse_transform(y_predtrain_SR)
y_predval_SR = scaler_sr.inverse_transform(y_predval_SR)
y_predtest_SR = scaler_sr.inverse_transform(y_predtest_SR)

In [ ]:
# Inverse scaling: SR_Org
Y_train_SR_Org = scaler_sr_org.inverse_transform(Y_train_SR_Org)
Y_val_SR_Org = scaler_sr_org.inverse_transform(Y_val_SR_Org)
y_test_SR_Org = scaler_sr_org.inverse_transform(y_test_SR_Org)
y_predtrain_SR_Org = scaler_sr_org.inverse_transform(y_predtrain_SR_Org)
y_predval_SR_Org = scaler_sr_org.inverse_transform(y_predval_SR_Org)
y_predtest_SR_Org = scaler_sr_org.inverse_transform(y_predtest_SR_Org)

In [ ]:
# Model Performance: Training
train_r2_SS = r2_score(Y_train_SS, y_predtrain_SS)
train_rmse_SS = np.sqrt(mean_squared_error(Y_train_SS, y_predtrain_SS))

train_r2_TTF = r2_score(Y_train_TTF, y_predtrain_TTF)
train_rmse_TTF = np.sqrt(mean_squared_error(Y_train_TTF, y_predtrain_TTF))

train_r2_SR = r2_score(Y_train_SR, y_predtrain_SR)
train_rmse_SR = np.sqrt(mean_squared_error(Y_train_SR, y_predtrain_SR))

train_r2_SR_Org = r2_score(Y_train_SR_Org, y_predtrain_SR_Org)
train_rmse_SR_Org = np.sqrt(mean_squared_error(Y_train_SR_Org, y_predtrain_SR_Org))

# Model Performance: Validation
val_r2_SS = r2_score(Y_val_SS, y_predval_SS)
val_rmse_SS = np.sqrt(mean_squared_error(Y_val_SS, y_predval_SS))

val_r2_TTF = r2_score(Y_val_TTF, y_predval_TTF)
val_rmse_TTF = np.sqrt(mean_squared_error(Y_val_TTF, y_predval_TTF))

val_r2_SR = r2_score(Y_val_SR, y_predval_SR)
val_rmse_SR = np.sqrt(mean_squared_error(Y_val_SR, y_predval_SR))

val_r2_SR_Org = r2_score(Y_val_SR_Org, y_predval_SR_Org)
val_rmse_SR_Org = np.sqrt(mean_squared_error(Y_val_SR_Org, y_predval_SR_Org))

# Model Performance: Testing
test_r2_SS = r2_score(y_test_SS, y_predtest_SS)
test_rmse_SS = np.sqrt(mean_squared_error(y_test_SS, y_predtest_SS))

test_r2_TTF = r2_score(y_test_TTF, y_predtest_TTF)
test_rmse_TTF = np.sqrt(mean_squared_error(y_test_TTF, y_predtest_TTF))

test_r2_SR = r2_score(y_test_SR, y_predtest_SR)
test_rmse_SR = np.sqrt(mean_squared_error(y_test_SR, y_predtest_SR))

test_r2_SR_Org = r2_score(y_test_SR_Org, y_predtest_SR_Org)
test_rmse_SR_Org = np.sqrt(mean_squared_error(y_test_SR_Org, y_predtest_SR_Org))

In [ ]:
# Print R2 Results
print("R2 scores: Train (SS) - %0.5f, Train (TTF) - %0.5f, Train (SR) - %0.5f, Train (SR_Org) - %0.5f" %(train_r2_SS, train_r2_TTF, train_r2_SR, train_r2_SR_Org))
print("R2 scores: Val (SS) - %0.5f, Val (TTF) - %0.5f, Val (SR) - %0.5f, Val (SR_Org) - %0.5f" %(val_r2_SS,val_r2_TTF,val_r2_SR, val_r2_SR_Org))
print("R2 scores: Test (SS) - %0.5f, Test (TTF) - %0.5f, Test (SR) - %0.5f, Test (SR_Org) - %0.5f" %(test_r2_SS,test_r2_TTF,test_r2_SR, test_r2_SR_Org))

In [ ]:
# Print RMSE Results
print("RMSE: Train (SS) - %0.5f, Train (TTF) - %0.5f, Train (SR) - %0.5f, Train (SR_Org) - %0.5f" %(train_rmse_SS, train_rmse_TTF, train_rmse_SR, train_rmse_SR_Org))
print("RMSE: Val (SS) - %0.5f, Val (TTF) - %0.5f, Val (SR) - %0.5f, Val (SR_Org) - %0.5f" %(val_rmse_SS, val_rmse_TTF, val_rmse_SR, val_rmse_SR_Org))
print("RMSE: Test (SS) - %0.5f, Test (TTF) - %0.5f, Test (SR) - %0.5f, Test (SR_Org) - %0.5f" %(test_rmse_SS, test_rmse_TTF, test_rmse_SR, test_rmse_SR_Org))

In [ ]:
# Overall plot:
ttime = df['Time'][n_steps:]
t1, testtime = train_test_split(ttime, test_size=0.2, shuffle=False)
traintime, valtime = train_test_split(t1, test_size=0.125, shuffle=False)

traintime = traintime[244:735]

fig = plt.figure(1, figsize=(20,6))
plt.plot(ttime, ydf_SS)
plt.plot(traintime, y_predtrain_SS)
plt.plot(valtime, y_predval_SS)
plt.plot(testtime, y_predtest_SS)
plt.xlabel('Time (s)')
plt.ylabel('Shear Stress (MPa)')
plt.legend(['Ground truth', 'Train Prediction', 'Validation Prediction', 'Test Prediction'])

fig = plt.figure(2, figsize=(20,6))
plt.plot(ttime, ydf_TTF)
plt.plot(traintime, y_predtrain_TTF)
plt.plot(valtime,  y_predval_TTF)
plt.plot(testtime, y_predtest_TTF)
plt.xlabel('Time (s)')
plt.ylabel('TTF (s)')
plt.legend(['Ground truth', 'Train Prediction', 'Validation Prediction', 'Test Prediction'])

ydf_SR = np.exp(ydf_SR)*10;
y_predtrain_SR = np.exp(y_predtrain_SR)*10;
y_predval_SR = np.exp(y_predval_SR)*10;
y_predtest_SR = np.exp(y_predtest_SR)*10;

fig = plt.figure(3, figsize=(20,6))
plt.plot(ttime, ydf_SR)
plt.plot(traintime, y_predtrain_SR)
plt.plot(valtime,  y_predval_SR)
plt.plot(testtime, y_predtest_SR)
plt.xlabel('Time (s)')
plt.ylabel('ln_V_V0')
plt.legend(['Ground truth', 'Train Prediction', 'Validation Prediction', 'Test Prediction'])

In [ ]:
# Extract Weights
names = [weight.name for layer in model.layers for weight in layer.weights]
weights = model.get_weights()

for name, weight in zip(names, weights):
    print(name, weight.shape)

In [ ]:
# Display Weights
model.get_weights()

In [ ]:
# Get scale
# print(scaler_ss.scale_)
# print(scaler_sr.scale_)
# print(scaler_A.scale_)
# print(scaler_ae.scale_)
r1 = scaler_ae.scale_.max()
r2 = scaler_A.scale_.max()
r3 = scaler_ss.scale_.max()
r4 = scaler_sr.scale_.max()
print(r1)
print(r2)
print(r3)
print(r4)

In [ ]:
# # Rescale constant C1
cols = ['C1']
data = pd.DataFrame(np.array([[0.07635498]]),columns=cols)
a = scaler_ss.inverse_transform(data)
Mu0 = a/20
print(Mu0)

In [ ]:
# # Rescale constant C2
cols = ['a']
data = pd.DataFrame(np.array([[0.72015053]]),columns=cols)
a = scaler_sr.inverse_transform(data)
a = a*r1
print(a)

In [ ]:
# # Rescale constant C3
cols = ['b']
data = pd.DataFrame(np.array([[0.11624802]]),columns=cols)
b = scaler_A.inverse_transform(data)
b = b*r1
print(b)

In [ ]:
# # Rescale constant C4
Dc= math.exp(1.6902565)
print(Dc)